In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

In [ ]:
!pip install nilearn

In [3]:
# 健常者グループの被験者IDのリスト
healthy_subject_ids = ['4101', '4103', '4106', '4107', '4109', '4110', '4111', '4112', '4113', '4114', '4115', '4116', '4117', '4118', '4119', '4120', '4121', '4122', '4123', '4125', '4126', '4128', '4129', '4130', '4131']

# パーキンソン病患者グループの被験者IDのリスト
patient_subject_ids = ['4201', '4204', '4205', '4206', '4207', '4208', '4210', '4211', '4212', '4213', '4214', '4215', '4217', '4218', '4219', '4220', '4221', '4224', '4225', '4226', '4227']

In [4]:
# sessionのリスト
ses_ids = ['1', '2']

In [5]:
# runsのリスト
run_ids = ['1', '2', '3', '4', '5', '6']

In [6]:
vs_titles = ["alerting", "executive", "orienting"]

In [7]:
import numpy as np
from scipy.stats import f

# ウェルチの検定
def oneway_ANOVA(*x, equal_var=False):
    k = len(x)
    ni = np.array([len(y) for y in x])
    mi = np.array([np.mean(y) for y in x])
    vi = np.array([np.var(y, ddof=1) for y in x])
    if equal_var:
        n = sum(ni)
        mean_x = sum(ni * mi) / n
        df1, df2 = k - 1, n - k
        F = sum(ni * (mi - mean_x)**2) / df1 / (sum((ni - 1) * vi) / df2)
    else:
        wi = ni / vi
        sum_wi = sum(wi)
        tmp = sum((1 - wi / sum_wi)**2 / (ni - 1)) / (k**2 - 1)
        m = sum(wi * mi) / sum_wi
        df1, df2 = k - 1, 1 / (3 * tmp)
        F = sum(wi * (mi - m)**2) / (df1 * (1 + 2 * (k - 2) * tmp))
    return F, f.sf(F, df1, df2)


In [8]:
from nilearn.input_data import NiftiMasker
from nilearn import datasets

def get_p_f_value(z_maps):
  # MNIアトラスの全脳マスクを取得
  mni = datasets.fetch_icbm152_2009()
  brain_mask = mni.mask
  masker = NiftiMasker(mask_img=brain_mask)
  z_map_lists = []
  for i in range(len(z_maps)):
    z_map_model = masker.fit_transform(z_maps[i])

    # ndarray->list
    z_map_lists.append(z_map_model.tolist()[0])

  # ボクセルごとにANOVAを実行
  f_value, p_value = oneway_ANOVA(z_map_lists)

  print('F値:', f_value)
  print('p値:', p_value)

In [ ]:
import numpy as np
import pandas as pd
from nilearn.glm.second_level import SecondLevelModel
from nilearn.plotting import plot_stat_map
import random
from nilearn.image import concat_imgs, mean_img

def compute_glm(contrast_maps, group_labels):
    design_matrix = pd.DataFrame({'intercept': np.ones(len(group_labels)), 'group': group_labels})
    second_level_model = SecondLevelModel(smoothing_fwhm=8)
    second_level_model = second_level_model.fit(contrast_maps, design_matrix=design_matrix)
    z_map = second_level_model.compute_contrast('group', output_type='z_score')
    return z_map

def get_contrast_maps(vs_title, subject_ids, ses_ids, run_ids):
    contrast_maps = []
    for subject_id in subject_ids:
        for ses_id in ses_ids:
            for run_id in run_ids:
                contrast_map_path = f'/content/drive/MyDrive/ANT/{vs_title}/sub-RC{subject_id}/ses-{ses_id}/sub-RC{subject_id}_ses-{ses_id}_task-ANT_run-{run_id}_bold.nii.gz'
                contrast_maps.append(contrast_map_path)
    return contrast_maps

def random_subset_average(vs_title, healthy_subject_ids, patient_subject_ids, ses_ids, run_ids, iterations=50):
    all_z_maps = []
    for i in range(iterations):
        random_healthy_subset = random.sample(healthy_subject_ids, len(healthy_subject_ids) // 2)
        random_patient_subset = random.sample(patient_subject_ids, len(patient_subject_ids) // 2)
        contrast_maps = get_contrast_maps(vs_title, random_healthy_subset + random_patient_subset, ses_ids, run_ids)
        z_map = compute_glm(contrast_maps, [0]*len(random_healthy_subset)*len(ses_ids)*len(run_ids)+[1]*len(random_patient_subset)*len(ses_ids)*len(run_ids))
        all_z_maps.append(z_map)

    # 4Dイメージとして統計マップを結合
    combined_stat_map = concat_imgs(all_z_maps)

    # グループ平均マップの計算
    return mean_img(combined_stat_map)

def random_subset_diff(vs_title):
  # 健常者群のランダムサブセットの平均Zマップを計算
  average_healthy_z_map = random_subset_average(vs_title, healthy_subject_ids, [], ses_ids, run_ids)

  # 健常者群のランダムサブセットとパーキンソン病患者群のランダムサブセットの平均の差のZマップを計算
  average_z_map = random_subset_average(vs_title, healthy_subject_ids, patient_subject_ids, ses_ids, run_ids)

  # 健常者群と患者群の間に特有の違いを抽出
  unique_group_difference_3d = average_z_map.get_fdata() - average_healthy_z_map.get_fdata()

  # Niftyに戻す
  group_z_map = nib.Nifti1Image(unique_group_difference_3d, average_z_map.affine)

  # 結果のプロット
  return group_z_map

In [ ]:
from nilearn.plotting import plot_stat_map

def get_diff():
  group_difference = {}
  for vs_title in vs_titles:
    z_map = random_subset_diff(vs_title)
    group_difference[vs_title] = z_map
    plot_stat_map(z_map, threshold=2.3, display_mode='z', cut_coords=5, black_bg=True, title=f'Unique difference between groups - {vs_title}')
  return group_difference

In [ ]:
# ランダム抽出の統計マップ平均の差（"健常者群＋パーキンソン病患者群" - "健常者群"）をN個取得する
N = 2
group_diffs = []
for i in range(N):
  group_diffs.append(get_diff())

In [ ]:
# そのN個のデータの各タイトル毎に、p値を求める
# p=1に近ければ近いほど、個人差を省いた均一化に成功している
for vs_title in vs_titles:
  diffs = []
  for i in range(N):
    diffs.append(group_diffs[i][vs_title])
  print(f'{vs_title}: ')
  get_p_f_value(diffs)
  print()

In [ ]:
from nilearn.glm import threshold_stats_img
def fpr(z_map, display):
  thresholded_map1, threshold1 = threshold_stats_img(
      z_map,
      alpha=0.001,
      height_control="fpr",
      cluster_threshold=10,
      two_sided=True,
  )

  # p<.001 の未補正しきい値マップ (10 ボクセルを超えるクラスターのみ)。
  plot_stat_map(
      thresholded_map1,
      cut_coords=display.cut_coords,
      threshold=threshold1,
      title="Thresholded z map, fpr <.001, clusters > 10 voxels",
  )

In [ ]:
from nilearn.glm import threshold_stats_img
def fdr(z_map, display):
  thresholded_map2, threshold2 = threshold_stats_img(
      z_map, alpha=0.05, height_control="fdr"
  )
  print(f"The FDR=.05 threshold is {threshold2:.3g}")

  plot_stat_map(
      thresholded_map2,
      cut_coords=display.cut_coords,
      title="Thresholded z map, expected fdr = .05",
      threshold=threshold2,
  )

In [ ]:
from nilearn.glm import threshold_stats_img
def fwer(z_map, display):
  # 3. FWER <.05 (ファミリーワイズエラー率) を使用し、クラスターレベルのしきい値は使用しない
  # データは集中的に平滑化されていないため、単純なボンフェローニ補正を使用。
  thresholded_map3, threshold3 = threshold_stats_img(
      z_map, alpha=0.05, height_control="bonferroni"
  )
  print(f"The p<.05 Bonferroni-corrected threshold is {threshold3:.3g}")

  plot_stat_map(
      thresholded_map3,
      cut_coords=display.cut_coords,
      title="Thresholded z map, expected fwer < .05",
      threshold=threshold3,
  )